In [1]:
import pandas as pd
import numpy as np
import os
from time import time
from tqdm import tqdm
from utilities.math_utils import RotationTranslationData
from utilities import data_utils
from glob import glob
import pickle

from time import time
%matplotlib inline

#### Params and Paths:

In [2]:
const_n_frames_per_side=100
const_shift=150

rot_n_frames_per_side=8
rot_shift=5

voxel_size=20
voting_scene_frac_to_drop = 0.3

In [3]:
base_dir = "../../OverfittedHackathon_data/voxelling_output/Test_world_2/vid_19/"
pickles_path = "../../OverfittedHackathon_data/voxelling_output/"
orig_files_path = "../../OverfittedHackathon_data/voxelling_output/Test/vid_19/"
labels_path = "../../OverfittedHackathon_data/voxelling_output/submission_files/test_vid_19_pred__lines/"
labels_path = "../../OverfittedHackathon_data/voxelling_output/submission_files/checks2/"

# labels_path = "voxelling_output/submission_files/checks/"

-----------------------------

In [4]:
def add_grid_and_hash(df, voxel_size):
    """ 
    add grid and hash to each point of a given point cloud
    """
    df_grid = df.iloc[:,:3]//voxel_size
    df_grid.iloc[:, :3] = df_grid.iloc[:, :3]*voxel_size + voxel_size//2
    df_grid['voxel_id'] = df_grid.apply(lambda x: hash((x[0], x[1], x[2])), axis=1)
    df_grid = pd.concat([df, df_grid], axis=1)
    df_grid.columns = ['x', 'y', 'z', 'r', 'x_grid', 'y_grid', 'z_grid', 'voxel_id']
    return df_grid

In [5]:
all_files = glob(base_dir+"/*point*")

In [6]:
vid = base_dir.split("/")[-2]

In [7]:
def create_list_dfs_voxel_scene(base_dir, voxel_size, upto=None):
    global all_files
    list_df_voxel_scene = []
#     all_files = glob(base_dir+"/*point*")
    if upto is not None:
        all_files = all_files[:upto]
    
    for f in tqdm(all_files):
        list_df_voxel_scene.append( add_grid_and_hash(pd.read_csv(f, header=None), voxel_size))
    list_df_voxel_scene = pd.Series(list_df_voxel_scene).values
    return list_df_voxel_scene

In [8]:
list_df_file = pickles_path+"list_df_voxel_scene__%s__voxel_size_%d.p"%(vid, voxel_size)

if os.path.exists(list_df_file):
    list_df_voxel_scene = pickle.load(open(list_df_file, "rb"))
else:
    list_df_voxel_scene = create_list_dfs_voxel_scene(base_dir, voxel_size, upto=None)
    pickle.dump(list_df_voxel_scene, open(pickles_path+"list_df_voxel_scene__%s__voxel_size_%d.p"%(vid, voxel_size), "wb"))

----------------------

In [9]:
def get_list_idx_for_frame(frame_idx, n_frames_per_side, shift):
    list_idx = np.array(list(np.arange(frame_idx-shift-n_frames_per_side,frame_idx-shift)) + list(np.arange(frame_idx+shift+1,frame_idx+shift+n_frames_per_side+1)))
    list_idx = [i for i in list_idx if i>=0 and i<=list_df_voxel_scene.shape[0]]
    return list_idx

In [10]:
def create_scene_voxel_df(list_df_voxel_scene, list_idx, frac_to_drop=0.5):
#     print(list_idx)
    list_df_voxel_scene_for_frame = list_df_voxel_scene[list_idx]
    df_scene = pd.concat(list_df_voxel_scene_for_frame)#.drop_duplicates("voxel_id")
#     print(df_scene)
#     df_scene.keys()
#     df_scene.groupby(["x_grid", "y_grid", "z_grid"]).value_counts()
    dfff = df_scene.groupby(["x_grid", "y_grid", "z_grid"]).apply(len).reset_index(drop=False)

    dfff = dfff[dfff[0]>frac_to_drop*len(list_idx)]
    dfff["v_index"] = dfff.iloc[:, :3].apply(lambda x: hash((x.iloc[0], x.iloc[1], x.iloc[2])), axis=1)
#     print(df_scene.shape)
    df_scene.drop_duplicates(["x_grid", "y_grid", "z_grid"], inplace=True)
#     print(df_scene.shape)
    df_scene = df_scene[df_scene.voxel_id.isin(dfff.v_index)]
#     print(df_scene.shape)
    
    return df_scene

In [11]:
def get_df_voxel_subtracted_frame(list_df_voxel_scene, frame_idx, n_frames_per_side, shift):
    
    list_idx_uf = get_list_idx_for_frame(frame_idx, n_frames_per_side, shift)
#     print ('--get_df_voxel_sub--\nlist idx_uf: {}\nlist_idx   :{}\n-------'.format(list_idx_uf, list_idx))
    df_voxel_scene = create_scene_voxel_df(list_df_voxel_scene, list_idx, frac_to_drop=voting_scene_frac_to_drop)
    df_voxel_frame = create_scene_voxel_df(list_df_voxel_scene, [frame_idx])

    df_subtracted_frame = df_voxel_frame[~df_voxel_frame.voxel_id.isin(df_voxel_scene.voxel_id)]
    return df_subtracted_frame

In [12]:
def point_to_voxel(ser, voxel_size):
    # ser: [x, y, z, r]
    ser_out = ser.iloc[:3]//voxel_size
    ser_out = ser_out*voxel_size + voxel_size//2
    voxel_id = hash((ser_out[0], ser_out[1], ser_out[2]))
    
#     return pd.Series(ser_out.tolist()+[voxel_id])
    return voxel_id

In [13]:
def save_frame_for_movie(frame, folder, all_files, frame_idx):
    if not os.path.exists(folder):
        os.mkdir(folder)
    filename = os.path.basename(all_files[frame_idx])
    frame.to_csv(folder + filename, header=None, index=False)
    labels_filename = filename.replace('pointcloud', 'labels')
    pd.DataFrame([0]*frame.shape[0]).to_csv(folder + labels_filename, header=None, index=False)

    egomotion_filename = filename.replace('pointcloud', 'egomotion')
    pd.DataFrame([0.]*6).T.to_csv(folder + egomotion_filename, header=None, index=False)
    print('frame {} saved successfuly'.format(frame_idx))

#### Filters:

In [14]:
def remove_distant_lines(frame_idx, df_labels, orig_files_path):
    filename = (7-len(str(frame_idx)))*"0"+str(frame_idx)+"_pointcloud.csv"
    dff = pd.read_csv(orig_files_path+filename, header=None)
    dff.columns = list("xyzr")
    dff["l"] = df_labels.tolist()
    dff = dff[dff.l==1]
#     print(dff.shape)
#     dff.head() 

#     dff = dff[dff.x>dff.x.min()+(dff.x.max()-dff.x.min())*0.2]
#     print(dff.shape)
#     dff.head()
    
    dff = dff[dff.z<(dff.z.min()+(dff.z.max()-dff.z.min())*0.3)]
#     print(dff.shape)
#     dff.head()

    dff = dff[(dff["r"]>5)&(dff["r"]<15)]
#     print(dff.shape)
#     dff.head()

    df_labels.loc[dff.index] = False
    return df_labels

In [15]:
def remove_distant_points(frame_idx, df_labels, orig_files_path):
    filename = (7-len(str(frame_idx)))*"0"+str(frame_idx)+"_pointcloud.csv"
    dff = pd.read_csv(orig_files_path+filename, header=None)
    dff.columns = list("xyzr")
    dff["l"] = df_labels.tolist()
    dff = dff[dff.l==1]
#     print(dff.shape)
#     dff.head() 
#     print(dff.describe())
    dff = dff[dff["x"]>5500]
#     print(dff.describe())
#     rr = np.sqrt(dff.x**2+dff.y**2)
#     dff = dff[rr<10]
#     print(dff.shape)
#     dff.head()
    
#     dff = dff[dff.z<dff.z.min()+(dff.z.max()-dff.z.min())*0.3]
#     print(dff.shape)
#     dff.head()

#     dff = dff[(dff["r"]>5)&(dff["r"]<15)]
#     print(dff.shape)
#     dff.head()

    df_labels.loc[dff.index] = False
    return df_labels

In [16]:
def remove_high_theta_points(frame_idx, df_labels, orig_files_path):
    filename = (7-len(str(frame_idx)))*"0"+str(frame_idx)+"_pointcloud.csv"
    dff = pd.read_csv(orig_files_path+filename, header=None)
    dff.columns = list("xyzr")
    dff["l"] = df_labels.tolist()
    dff = dff[dff.l==1]
#     print(dff.shape)
#     dff.head() 

    theta = np.abs(np.arctan(dff.y/dff.x))
    dff = dff[theta>0.97*theta.max()]
    dff = dff[dff.x>300]
#     print(dff.shape)
#     dff.head()
    
#     dff = dff[dff.z<dff.z.min()+(dff.z.max()-dff.z.min())*0.3]
#     print(dff.shape)
#     dff.head()

#     dff = dff[(dff["r"]>5)&(dff["r"]<15)]
#     print(dff.shape)
#     dff.head()

    df_labels.loc[dff.index] = False
    return df_labels

In [17]:
def remove_close_lines(frame_idx, df_labels, orig_files_path):
    filename = (7-len(str(frame_idx)))*"0"+str(frame_idx)+"_pointcloud.csv"
    dff = pd.read_csv(orig_files_path+filename, header=None)
    dff.columns = list("xyzr")
    dff["l"] = df_labels.tolist()
    dff = dff[dff.l==1]
#     print(dff.shape)
#     dff.head() 

    dff = dff[dff.x<dff.x.min()+(dff.x.max()-dff.x.min())*0.1]
#     print(dff.shape)
#     dff.head()
    
    dff = dff[dff.z<dff.z.min()+(dff.z.max()-dff.z.min())*0.01]
#     print(dff.shape)
#     dff.head()

#     dff = dff[(dff["r"]>5)&(dff["r"]<15)]
# #     print(dff.shape)
# #     dff.head()

    df_labels.loc[dff.index] = False
    return df_labels

In [18]:
def remove_high_points(frame_idx, df_labels, orig_files_path):
    filename = (7-len(str(frame_idx)))*"0"+str(frame_idx)+"_pointcloud.csv"
    dff = pd.read_csv(orig_files_path+filename, header=None)
    dff.columns = list("xyzr")
    dff["l"] = df_labels.tolist()
    dff = dff[dff.l==1]
#     print(dff.shape)
#     dff.head() 

#     dff = dff[dff.x<dff.x.min()+(dff.x.max()-dff.x.min())*0.15]
#     print(dff.shape)
#     dff.head()
    
    dff = dff[dff.z>400]
    print(dff.shape)
    dff.head()

#     dff = dff[(dff["r"]>5)&(dff["r"]<15)]
# #     print(dff.shape)
# #     dff.head()

    df_labels.loc[dff.index] = False
    return df_labels

In [19]:
def remove_low_points(frame_idx, df_labels, orig_files_path):
    filename = (7-len(str(frame_idx)))*"0"+str(frame_idx)+"_pointcloud.csv"
    dff = pd.read_csv(orig_files_path+filename, header=None)
    dff.columns = list("xyzr")
    dff["l"] = df_labels.tolist()
    dff = dff[dff.l==1]
#     print(dff.shape)
#     dff.head() 

#     dff = dff[dff.x<dff.x.min()+(dff.x.max()-dff.x.min())*0.15]
#     print(dff.shape)
#     dff.head()
#     print("\n\nXMIN:", dff.x.min())
    dff = dff[dff.z<dff.z.min()+15]
    print(dff.shape)
    dff.head()

#     dff = dff[(dff["r"]>5)&(dff["r"]<15)]
# #     print(dff.shape)
# #     dff.head()

    df_labels.loc[dff.index] = False
    return df_labels

In [22]:
def remove_low_points_2(frame_idx, df_labels, orig_files_path):
    filename = (7-len(str(frame_idx)))*"0"+str(frame_idx)+"_pointcloud.csv"
    dff = pd.read_csv(orig_files_path+filename, header=None)
    dff.columns = list("xyzr")
    dff["l"] = df_labels.tolist()
    dff = dff[dff.l==1]
#     print(dff.shape)
#     dff.head() 

#     dff = dff[dff.x<dff.x.min()+(dff.x.max()-dff.x.min())*0.15]
#     print(dff.shape)
#     dff.head()
#     print("\n\nXMIN:", dff.x.min())
    dff = dff[dff.x>dff.x.min()+300]
#     dff = dff[dff.z<(dff.z.min()+40)]
    dff = dff[dff.z<(dff.z.nsmallest(1000).iloc[999]+10)]

    print(dff)
        

#     dff = dff[(dff["r"]>5)&(dff["r"]<15)]

    print(dff.shape)
    dff.head()

#     dff = dff[(dff["r"]>5)&(dff["r"]<15)]
# #     print(dff.shape)
# #     dff.head()

    df_labels.loc[dff.index] = False
    return df_labels

---------------------------------

In [23]:
# frame_idx=15
frames_range = range(1, 3)# range(1000)
frames_range = range(420, 431)
# frames_range = range(15,900)
for frame_idx in tqdm(frames_range):
    print("=========================================================================")
    print("frame_idx = %d"%frame_idx)
    print("=========================================================================")

    tic = time()
    
    shift = const_shift
    n_frames_per_side = const_n_frames_per_side
    
    list_idx_uf = get_list_idx_for_frame(frame_idx, n_frames_per_side, shift)
    list_idx = [i for i in list_idx_uf if i>=0 and i<=list_df_voxel_scene.shape[0]-1]
#     print ('--main--\nlist idx_uf: {}\nlist_idx   :{}\n-------'.format(list_idx_uf, list_idx))
    toc = time(); print(toc-tic, ": list_idx = get_list_idx_for_frame(frame_idx, n_frames_per_side, shift)"); tic=time()
    
    df_scene = create_scene_voxel_df(list_df_voxel_scene, list_idx)
    
    
#     if frame_idx%frames_per_scene==0 :
    df_subtracted_frame = get_df_voxel_subtracted_frame(list_df_voxel_scene, frame_idx, n_frames_per_side, shift)
    toc = time(); print(toc-tic, ": df_subtracted_frame = get_df_voxel_subtracted_frame(list_df_voxel_scene, frame_idx, n_frames_per_side, shift)"); tic=time()
    
    df_frame_orig = pd.read_csv(all_files[frame_idx], header=None)
#     df_frame_orig["voxel_id"] =  df_frame_orig.apply(lambda x: point_to_voxel(x, voxel_size), axis=1)
    df_frame_orig["voxel_id"] =  ((df_frame_orig.iloc[:, :3]//voxel_size)*voxel_size + voxel_size//2).apply(lambda x: hash((x[0], x[1], x[2])), axis=1)
    toc = time(); print(toc-tic, ": df_frame_orig['voxel_id'] =  df_frame_orig.apply(lambda x: point_to_voxel(x, voxel_size), axis=1)"); tic=time()
    
    df_labels = df_frame_orig.voxel_id.isin(df_subtracted_frame.voxel_id)
    toc = time(); print(toc-tic, ": df_labels = df_frame_orig.voxel_id.isin(df_subtracted_frame.voxel_id)"); tic=time()
    
    if (df_labels.sum()/df_labels.shape[0])>0.5:
        print("\nIN ROTATION MODE! fraction of green pixels before:", (df_labels.sum()/df_labels.shape[0]))
        n_frames_per_side=rot_n_frames_per_side
        shift=rot_shift
        
        list_idx_uf = get_list_idx_for_frame(frame_idx, n_frames_per_side, shift)
        list_idx = [i for i in list_idx_uf if i>=0 and i<=list_df_voxel_scene.shape[0]-1]
        print("\nIN ROTATION MODE: list_idx:", list_idx)
    #     print ('--main--\nlist idx_uf: {}\nlist_idx   :{}\n-------'.format(list_idx_uf, list_idx))
        toc = time(); print(toc-tic, ": list_idx = get_list_idx_for_frame(frame_idx, n_frames_per_side, shift)"); tic=time()

        df_scene = create_scene_voxel_df(list_df_voxel_scene, list_idx)


    #     if frame_idx%frames_per_scene==0 :
        df_subtracted_frame = get_df_voxel_subtracted_frame(list_df_voxel_scene, frame_idx, n_frames_per_side, shift)
        toc = time(); print(toc-tic, ": df_subtracted_frame = get_df_voxel_subtracted_frame(list_df_voxel_scene, frame_idx, n_frames_per_side, shift)"); tic=time()

        df_frame_orig = pd.read_csv(all_files[frame_idx], header=None)
    #     df_frame_orig["voxel_id"] =  df_frame_orig.apply(lambda x: point_to_voxel(x, voxel_size), axis=1)
        df_frame_orig["voxel_id"] =  ((df_frame_orig.iloc[:, :3]//voxel_size)*voxel_size + voxel_size//2).apply(lambda x: hash((x[0], x[1], x[2])), axis=1)
        toc = time(); print(toc-tic, ": df_frame_orig['voxel_id'] =  df_frame_orig.apply(lambda x: point_to_voxel(x, voxel_size), axis=1)"); tic=time()

        df_labels = df_frame_orig.voxel_id.isin(df_subtracted_frame.voxel_id)
        toc = time(); print(toc-tic, ": df_labels = df_frame_orig.voxel_id.isin(df_subtracted_frame.voxel_id)"); tic=time()

        print("\nIN ROTATION MODE! fraction of green pixels after:", (df_labels.sum()/df_labels.shape[0]))
        
    
    
    # filters
    print(df_labels.sum())
    df_labels = remove_distant_lines(frame_idx, df_labels, orig_files_path)
    print(df_labels.sum())
    df_labels = remove_distant_points(frame_idx, df_labels, orig_files_path)
    print(df_labels.sum())
    df_labels = remove_close_lines(frame_idx, df_labels, orig_files_path)
    print(df_labels.sum())
    df_labels = remove_high_points(frame_idx, df_labels, orig_files_path)
    print(df_labels.sum())
    df_labels = remove_low_points(frame_idx, df_labels, orig_files_path)
    print("LOW POINTS:", df_labels.sum())
    df_labels = remove_low_points_2(frame_idx, df_labels, orig_files_path)
    print("LOW POINTS 2:", df_labels.sum())
    df_labels = remove_high_theta_points(frame_idx, df_labels, orig_files_path)
    print(df_labels.sum())
    
    
#     df_labels.iloc[:] = False
    
    df_frame_orig_subtracted = df_frame_orig[df_labels]
    toc = time(); print(toc-tic, ": df_frame_orig_subtracted = df_frame_orig[df_labels]"); tic=time()
    
    print(df_frame_orig_subtracted.shape)
    df_labels = df_labels.astype(int)
    file_labels = labels_path + os.path.basename(all_files[frame_idx]).replace("pointcloud", "labels")
    print("file_labels:", file_labels)
    df_labels.to_csv(file_labels, header=None, index=False)
    toc = time(); print(toc-tic, ': df_labels.to_csv(pickles_path+"df_labels__frame_%d__%s__voxel_size_%d_(%d,%d,%d).p"%(frame_idx, vid, voxel_size, n_frames_per_side, shift, n_frames_per_side))'); tic=time()
    
    print(df_labels.sum())
    df_frame_orig_subtracted = df_frame_orig_subtracted.iloc[:, :4]
#     df_frame_orig_subtracted.to_csv(pickles_path+"df_frame_orig_subtracted__frame_%d__%s__voxel_size%d_(%d,%d,%d).p"%(frame_idx, vid, voxel_size, n_frames_per_side, shift, n_frames_per_side)), header=None, index=False)
#     save_frame_for_movie(df_frame_orig_subtracted, "tmp_only_labeled/", all_files, frame_idx)
    toc = time(); print(toc-tic, ': save_frame_for_movie(df_frame_orig_subtracted, "tmp_only_labeled/", all_files, frame_idx)'); tic=time()


  0%|                                                                                                                                                                           | 0/11 [00:00<?, ?it/s]

frame_idx = 420
0.0009965896606445312 : list_idx = get_list_idx_for_frame(frame_idx, n_frames_per_side, shift)
8.83808445930481 : df_subtracted_frame = get_df_voxel_subtracted_frame(list_df_voxel_scene, frame_idx, n_frames_per_side, shift)
1.0558745861053467 : df_frame_orig['voxel_id'] =  df_frame_orig.apply(lambda x: point_to_voxel(x, voxel_size), axis=1)
0.0 : df_labels = df_frame_orig.voxel_id.isin(df_subtracted_frame.voxel_id)
18895
16981
15512
13678
(401, 5)
13277
(860, 5)
LOW POINTS: 12417
          x     y    z   r     l
12834  5392  3686 -158  21  True
12835  5395  3667 -158  20  True
12836  5413  3656 -159  19  True
12837  5444  3654 -160  20  True
13064  5449 -1532 -160   2  True
13065  5423 -1547 -159   2  True
13066  5340 -1546 -157  16  True
14035  3711  2522 -156  22  True
14036  3711  2507 -156  19  True
14037  3711  2491 -156  18  True
14038  3711  2476 -156  18  True
14039  3711  2460 -156  19  True
14040  3711  2445 -156  20  True
14045  3711  2367 -156  18  True
1404


  9%|██████████████▊                                                                                                                                                    | 1/11 [00:10<01:42, 10.24s/it]

frame_idx = 421
0.0009963512420654297 : list_idx = get_list_idx_for_frame(frame_idx, n_frames_per_side, shift)
10.07064962387085 : df_subtracted_frame = get_df_voxel_subtracted_frame(list_df_voxel_scene, frame_idx, n_frames_per_side, shift)
1.291712999343872 : df_frame_orig['voxel_id'] =  df_frame_orig.apply(lambda x: point_to_voxel(x, voxel_size), axis=1)
0.001993417739868164 : df_labels = df_frame_orig.voxel_id.isin(df_subtracted_frame.voxel_id)
19591
17655
16094
14368
(454, 5)
13914
(757, 5)
LOW POINTS: 13157
          x     y    z   r     l
13074  5398 -1495 -159  15  True
13075  5431 -1527 -160   2  True
13076  5405 -1542 -159   2  True
13222  4909  3829 -165  25  True
13223  4924  3822 -165  25  True
13224  4939  3813 -166  25  True
13225  4958  3807 -166  25  True
13226  4982  3804 -167  26  True
13227  5006  3802 -168  23  True
13228  5030  3799 -169  23  True
13243  5345  3700 -179  20  True
13244  5348  3679 -179  18  True
14455  3684  2365 -170  15  True
14456  3684  2349 -1


 18%|█████████████████████████████▋                                                                                                                                     | 2/11 [00:21<01:38, 11.00s/it]

...


 82%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                             | 9/11 [02:08<00:28, 14.24s/it]

frame_idx = 429
0.0 : list_idx = get_list_idx_for_frame(frame_idx, n_frames_per_side, shift)
8.986368179321289 : df_subtracted_frame = get_df_voxel_subtracted_frame(list_df_voxel_scene, frame_idx, n_frames_per_side, shift)
1.067859411239624 : df_frame_orig['voxel_id'] =  df_frame_orig.apply(lambda x: point_to_voxel(x, voxel_size), axis=1)
0.0 : df_labels = df_frame_orig.voxel_id.isin(df_subtracted_frame.voxel_id)

IN ROTATION MODE! fraction of green pixels before: 0.6386338863994186

IN ROTATION MODE: list_idx: [416, 417, 418, 419, 420, 421, 422, 423, 435, 436, 437, 438, 439, 440, 441, 442]
0.0 : list_idx = get_list_idx_for_frame(frame_idx, n_frames_per_side, shift)
2.651576280593872 : df_subtracted_frame = get_df_voxel_subtracted_frame(list_df_voxel_scene, frame_idx, n_frames_per_side, shift)
1.0723609924316406 : df_frame_orig['voxel_id'] =  df_frame_orig.apply(lambda x: point_to_voxel(x, voxel_size), axis=1)
0.0 : df_labels = df_frame_orig.voxel_id.isin(df_subtracted_frame.voxel_id)



 91%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎              | 10/11 [02:22<00:14, 14.22s/it]

frame_idx = 430
0.0 : list_idx = get_list_idx_for_frame(frame_idx, n_frames_per_side, shift)
9.200546741485596 : df_subtracted_frame = get_df_voxel_subtracted_frame(list_df_voxel_scene, frame_idx, n_frames_per_side, shift)
1.272355318069458 : df_frame_orig['voxel_id'] =  df_frame_orig.apply(lambda x: point_to_voxel(x, voxel_size), axis=1)
0.0 : df_labels = df_frame_orig.voxel_id.isin(df_subtracted_frame.voxel_id)

IN ROTATION MODE! fraction of green pixels before: 0.6538711776187378

IN ROTATION MODE: list_idx: [417, 418, 419, 420, 421, 422, 423, 424, 436, 437, 438, 439, 440, 441, 442, 443]
0.007979393005371094 : list_idx = get_list_idx_for_frame(frame_idx, n_frames_per_side, shift)
3.0808377265930176 : df_subtracted_frame = get_df_voxel_subtracted_frame(list_df_voxel_scene, frame_idx, n_frames_per_side, shift)
1.2429955005645752 : df_frame_orig['voxel_id'] =  df_frame_orig.apply(lambda x: point_to_voxel(x, voxel_size), axis=1)
0.001983165740966797 : df_labels = df_frame_orig.voxel_id.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [02:37<00:00, 14.31s/it]
